In [0]:
%pip install openai unidecode gspread==5.12.4 tiktoken mlflow
%restart_python
%load_ext autoreload
%autoreload 2 

In [0]:
from abc import ABC, abstractmethod
import time
import mlflow
from contextlib import contextmanager
import json
import pandas as pd
import datetime
import re
import openai
from openai import OpenAI
import gspread
import random
import logging
import os
from unidecode import unidecode
from pathlib import Path
import pyspark
from pyspark.sql.functions import *
from functools import reduce
from typing import *
import tiktoken
import sys


# Add the parent directory of 'localizers' to sys.path
project_root = Path('/Workspace/Users/krista@jamcity.com/CentralizedLocalizationWorkflow/localizers')
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

### Localizer Modules 
from general_config import *
from ml_tracker import *
from generic_localizer import *


In [0]:
%run "./authenticationScript"

In [0]:
#old "https://docs.google.com/spreadsheets/d/12H9coNcohbMQulaWBjM3LFN7M9A4djE6zni7hyZn6hA/edit?gid=0#gid=0"

In [0]:
#new: "https://docs.google.com/spreadsheets/d/1Mst0Jwy-yDO50_8LdSIJHl1uMJcx2AbxSJ-4Kuplmdg/edit?gid=0#gid=0"

In [0]:
#setup_widgets()

In [0]:
#TODO update authenticatinon script and secrets!!
gsheet_client = get_gspread_client_from_secret_old()
gpt_client = get_model_client() 

In [0]:
# Get request from widgets
request = get_request()

# Generic Configuration
cfg = {
    "input": {"required_tabs": ["input","output"]},
    "char_limit_policy": "loose",
}

In [0]:
request

In [0]:
# Purpose is to take an input with some explicitly known columns ['en_US','char_limit'(optional),'context', 'token','other context']

In [0]:
localizer = GenericLocalizer(request = request, 
                            gsheet_client =gsheet_client, 
                            gpt_client = gpt_client, 
                            cfg=cfg)

In [0]:
inputs = localizer.load_inputs()


In [0]:
prepped = localizer.preprocess(inputs)

In [0]:
groups = localizer.build_prompts(prepped)

In [0]:
#groups['Spanish (Latin America)']

In [0]:
raw_results = localizer.translate(groups)

In [0]:
processed = localizer.postprocess(raw_results)

In [0]:
localizer.write_outputs(processed)

In [0]:
processed_groups = dict(zip(localizer.lang_cds, processed))

In [0]:
processed[0]

In [0]:
starting = localizer.df.copy()
#for i in 
for lang,df in processed_groups.items():
    starting = starting.merge(df[['row_idx',lang]],on = ["row_idx"], how='left')

In [0]:
starting

In [0]:
starting = localizer.df.copy()
#for i in 
for i in processed:
    cols = i.columns
    for j in cols:
        if j in localizer.lang_cds:
            select_col = j    
    starting = starting.merge(i[['row_idx',select_col]],on = ["row_idx"], how='left')

In [0]:
starting

In [0]:
results = localizer.run()

In [0]:
dbutils.notebook.exit(json.dumps(results))